# Machine Learning and Statistics: Hands On 1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

## Model Building

We will build models for a triggered photon counter with a fixed exposure time. Assume that the number of photons $n$ entering the detector follows a [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution):
$$
P(n\mid \lambda) = e^{-\lambda}\, \frac{\lambda^n}{n!}
$$
You will first create a **probabilistic model** for this photon counter, i.e., code to evaluate the likelihood $P(n\mid \lambda)$.

### Exercise 1

Fill in the function below to calculate $\log P(n\mid \lambda)$ without using any statistical libraries.

Notes:
 - `lambda` is a reserved keyword in python, so we use `lam` instead.
 - Use [scipy.special.gammaln](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gammaln.html) to evaluate $\log n! = \log\Gamma(n + 1)$.

In [ ]:
import scipy.special

In [ ]:
def log_likelihood(n, lam):
    assert n >= 0
    # Add your code here...

Check that your probabilities are correctly normalized:

In [ ]:
for lam in 0.5, 1., 2.:
    probs = [np.exp(log_likelihood(n, lam)) for n in range(100)]
    print(f'lam={lam}, sum of probs={np.sum(probs):.5f}')

Now reimplement your function to use [scipy.stats.poisson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html):

In [ ]:
def log_likelihood(n, lam):
    assert n >= 0
    # Add your code here...

Again, check that your probabilities are correctly normalized:

In [ ]:
for lam in 0.5, 1., 2.:
    probs = [np.exp(log_likelihood(n, lam)) for n in range(100)]
    print(f'lam={lam}, sum of probs={np.sum(probs):.5f}')

Finally, modify your model to account for your counter's trigger, which only outputs a value of $n$ when at least one photon is detected $n \ge 1$.

In [ ]:
def log_likelihood(n, lam):
    assert n >= 1
    # Add your code here...

Check that your probabilities are correctly normalized for $n \ge 1$:

In [ ]:
for lam in 0.5, 1., 2.:
    probs = [np.exp(log_likelihood(n, lam)) for n in range(1, 100)]
    print(f'lam={lam}, sum of probs={np.sum(probs):.5f}')

### Exercise 2

Next, build a corresponding **generative model** by completing the function below.

Notes:
 - Use `scipy.stats.poisson` again.
 - You should never return $n=0$ since you are simulating a **triggered** counter.

In [ ]:
def generate(lam, random_state):
    """Generative model of triggered photon counter.
    
    Parameters
    ----------
    lam : float
        Mean number of photons received during an exposure (without a trigger).
    random_state : numpy random_state object
        Random state to use for reproducible random numbers.
        
    Returns
    -------
    int
        Number of generated photons.
    """
    # Add your code here...

Generate some random samples:

In [ ]:
lam=2
random_state = np.random.RandomState(seed=123)
samples = np.array([generate(lam, random_state) for sample in range(10000)])

Check that your trigger is working:

In [ ]:
assert np.all(samples > 0)

Check that your samples have the expected mean:

In [ ]:
assert np.allclose(np.mean(samples), lam / (1 - np.exp(-lam)), atol=1e-2)

## Gaussian Mixtures

Read some of the overlapping sources data we generated earlier:

In [ ]:
sources_4 = pd.read_hdf('data/sources_4_xy.hf5')
sources_5 = pd.read_hdf('data/sources_5_xy.hf5')
sources_8 = pd.read_hdf('data/sources_8_xy.hf5')

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(8, 3))
for col, data in enumerate((sources_4, sources_5, sources_8)):
    ax[col].scatter(data.iloc[:, 0], data.iloc[:, 1], s=5, lw=0)

Recall that we can fit a mixture of Gaussians to each of these datasets, e.g.

In [ ]:
from sklearn import mixture

In [ ]:
fit = mixture.GaussianMixture(n_components=2).fit(sources_5)

To display the fit results:

In [ ]:
from mls import display_gmm

In [ ]:
display_gmm(sources_5, fit)

This is all fine if you know in advance how many sources the data contains, but you usually don't for interesting problems.

A popular method for selecting the "best" number of Gaussian is to use the [Akaike information criterion (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion) and [Bayesian information criterion (BIC)](https://en.wikipedia.org/wiki/Bayesian_information_criterion). These are different measures of the "goodness of fit" of the GMM to the data.  They both reward small residuals while penalizing extra fit parameters. However, both methods are ad-hoc and you should prefer (much more expensive) [Bayesian model selection methods](https://github.com/dkirkby/MachineLearningStatistics/blob/master/notebooks/ModelSelection.ipynb) when the answer really matters, or you need uncertainty estimates.

### Exercise 3

Use the methods `fit.aic()` and `fit.bic()` to plot the AIC and BIC for `n_components=1,2,3,4` with each of these three datasets.

In [ ]:
def plot_criteria(data):
    # Add your code here...

In [ ]:
plot_criteria(sources_4)

In [ ]:
plot_criteria(sources_5)

In [ ]:
plot_criteria(sources_8)

## Expectation Maximization

The [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation-maximization_algorithm) is used to implement many machine learning methods, including several we have already studied: K-means, factor analysis and weighted PCA.

The basic idea of EM is to optimize a goal function that depends on two disjoint sets of parameters by alternately updating one set and then the other, using a scheme that is guaranteed to improve the goal function (although generally to a local rather than global optimum). The alternating updates are called the E-step and M-step.

The K-means is one of the simplest uses of EM, so is a good way to get some hands-on experience.

### Exercise 4

Implement the function below to perform a K-means E-step. Hint: you might find [np.argmin](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argmin.html) useful.

In [ ]:
def E_step(X, centers):
    """Perform a K-means E-step.
    
    Assign each sample to the cluster with the nearest center, using the
    Euclidean norm to measure distance between a sample and a cluster center.
    
    Parameters
    ----------
    X : array with shape (N, D)
        Input data consisting of N samples in D dimensions.
    centers : array with shape (n, D)
        Centers of the the n clusters in D dimensions.
        
    Returns
    -------
    integer array with shape (N,)
        Cluster index of each sample, in the range 0 to n-1.
    """
    N, D = X.shape
    n = len(centers)
    assert centers.shape[1] == D
    indices = np.empty(N, int)
    # Add your code here...

    return indices

In [ ]:
# A correct solution should pass these tests.
gen = np.random.RandomState(seed=123)
X = gen.normal(size=(20, 2))
centers = np.array([[0., 0.], [0., 10.]])
X[50:] += centers[1]
indices = E_step(X, centers)
assert np.all(indices[:50] == 0)
assert np.all(indices[50:] == 1)

centers = gen.uniform(size=(5, 2))
indices = E_step(X, centers)
assert np.array_equal(indices, [4, 1, 4, 4, 1, 0, 1, 0, 2, 1, 2, 4, 0, 1, 0, 1, 0, 1, 4, 4])

Next, implement the function below to perform a K-means M-step:

In [ ]:
def M_step(X, indices, n):
    """Perform a K-means M-step.
    
    Calculate the center of each cluster as the geometric mean of its assigned samples.
    
    The centers of any clusters without any assigned samples should be set to the origin.
    
    Parameters
    ----------
    X : array with shape (N, D)
        Input data consisting of N samples in D dimensions.
    indices : integer array with shape (N,)
        Cluster index of each sample, in the range 0 to n-1.
    n : int
        Number of clusters.  Must be <= N.
        
    Returns
    -------
    array with shape (n, D)
        Centers of the the n clusters in D dimensions.
    """
    N, D = X.shape
    assert indices.shape == (N,)
    assert n <= N
    centers = np.zeros((n, D))
    # Add your code here...
    
    return centers

In [ ]:
# A correct solution should pass these tests.
gen = np.random.RandomState(seed=123)
X = np.ones((20, 2))
indices = np.zeros(20, int)
centers = M_step(X, indices, 1)
assert np.all(centers == 1.)

n = 5
indices = gen.randint(n, size=len(X))
centers = M_step(X, indices, n)
assert np.all(centers == 1.)

X = gen.uniform(size=X.shape)
centers = M_step(X, indices, n)
assert np.allclose(
    np.round(centers, 3),
    [[ 0.494,  0.381], [ 0.592,  0.645],
     [ 0.571,  0.371], [ 0.234,  0.634],
     [ 0.250,  0.386]])

You have now implemented the core of the K-means algorithm.  Experiment with it using this simple wrapper, which makes a scatter plot of the first two columns after each iteration. The actual sklearn wrapper combines the result of several random starting points and has other refinements.

In [ ]:
def KMeans_fit(data, n_clusters, nsteps, seed=123):
    X = data.values
    N, D = X.shape
    assert n_clusters <= N
    gen = np.random.RandomState(seed=seed)
    # Pick random samples as the initial centers.
    shuffle = gen.permutation(N)
    centers = X[shuffle[:n_clusters]]
    # Perform an initial E step to assign samples to clusters.
    indices = E_step(X, centers)
    # Loop over iterations.
    for i in range(nsteps):
        centers = M_step(X, indices, n_clusters)
        indices = E_step(X, centers)
    # Plot the result.
    cmap = np.array(sns.color_palette())
    plt.scatter(X[:, 0], X[:, 1], c=cmap[indices % len(cmap)])
    plt.scatter(centers[:, 0], centers[:, 1], marker='+', c='k', s=400, lw=5)
    plt.show()

Try this out on some randomly generated 2D data with 3 separate clusters (using the handy [make_blobs](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html)):

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
gen = np.random.RandomState(seed=123)
X, _ = make_blobs(500, 2, [[-3,-3],[0,3],[3,-3]], random_state=gen)
data = pd.DataFrame(X, columns=('x0', 'x1'))

For this simple test, you should find a good solution after two iterations:

In [ ]:
KMeans_fit(data, n_clusters=3, nsteps=0);
KMeans_fit(data, n_clusters=3, nsteps=1);
KMeans_fit(data, n_clusters=3, nsteps=2);

## Clustering in Many Dimensions

We previously looked for clusters where each sample $(x,y)$ was a point in 2D.

What if we treat a whole image as one sample in 256 dimensions (one per pixel)?  Can we find clusters of images in this large dimension?

Recall the parameters of the generative model:
 - Number of sources $N$ = 1-4.
 - $(x_0, y_0, \sigma, e_1, e_2)$ for each source.
 
What might clusters of images correlate with from this list?

To simplify the problem, we will focus on the size ($\sigma$) of images with a single isolated cluster ($N=1$):

In [ ]:
df_img = pd.read_hdf('data/sources_img.hf5')
df_par = pd.read_hdf('data/sources_par.hf5')
isolated = df_par['num_source'] == 1
df_iso = df_img[isolated]
sigma = np.concatenate(df_par[isolated]['sigma'].values)

Use the following function to compare the size distributions of isolated sources assigned to different clusters:

In [ ]:
def compare_sizes(fit):
    for k in range(fit.n_clusters):
        plt.hist(sigma[fit.labels_ == k], bins=30, range=(0, 3.5),
                 density=True, histtype='step', lw=2, label=f'Cluster #{k}')
    plt.legend()
    return fit

For example:

In [ ]:
from sklearn import cluster

In [ ]:
fit = compare_sizes(cluster.KMeans(n_clusters=3, random_state=1).fit(df_iso))

In this example, all 3 clusters have essentially the same distribution of sizes.

### Exercise 5

Look for 10 clusters instead of 3 (again using `random_state=1`). One cluster should be clearly different from the others.

In [ ]:
fit = compare_sizes(cluster.KMeans(n_clusters=10, random_state=1).fit(df_iso))

The `plot_image()` function displays a single image:

In [ ]:
from mls import plot_image

In [ ]:
help(plot_image)

For example:

In [ ]:
plot_image(df_iso.iloc[4].values, label='#4')

Write code to identify and plot some members of this cluster, and check that they make sense:

In [ ]:
# Add your code here...

Repeat the cluster finding with some different seed values for `random_state`. How robust is this cluster of large sizes?

In general, it is difficult to visualize clusters (or anything) in more than 2 dimensions. Your options are to:
- Use attributes such as color and size to display additional dimensions.
- Use 3D visualizations.
- Use an embedding transformation to map high dimensional data to 2D or 3D.

Try the embedding approach by uploading our 256-dimensional data to http://projector.tensorflow.org/. You will first need to save the data in the "tab-separated value" (TSV) format. We will only save the first 1000 images to keep the file size small:

In [ ]:
df_iso[:1000].to_csv('data/df_iso.tsv', sep='\t', index=False, header=False)

Experiment with the PCA visualization option, which reduces the dimensionality to 2 or 3 dimensions.

You can also experiment with the [tSNE](https://distill.pub/2016/misread-tsne/) visualizations. These can be hypnotic but I have not found them to be very enlightening for scientific data.